In [2]:
%load_ext autoreload
%autoreload 2
import os
import torch
import json
import numpy as np
from tqdm import tqdm
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


/workspaces/Document_Layout_Understanding/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [ ]:
class FUNSDDataset(Dataset):
    def __init__(self, data_path, processor, max_length=256):
        with open(data_path, 'r') as f:
            self.examples = json.load(f)
        
        self.processor = processor
        self.max_length = max_length
        
        all_labels = set()
        for example in self.examples:
            all_labels.update(example['labels'])
        
        self.label2id = {label: idx for idx, label in enumerate(sorted(all_labels))}
        self.id2label = {idx: label for label, idx in self.label2id.items()}
        
        print(f"Loaded {len(self.examples)} documents")
        print(f"Found {len(self.label2id)} unique labels")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        
        image = Image.open(example['image_path']).convert("RGB")
        
        words = example['words']
        boxes = example['bboxes']
        
        labels = [self.label2id[label] for label in example['labels']]

        encoding = self.processor(
            image,
            words,
            boxes=boxes,
            word_labels=labels,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        for key, val in encoding.items():
            encoding[key] = val.squeeze(0)
            
        return encoding

In [ ]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

train_data_path = "../data/processed/FUNSD/training.json"
train_dataset = FUNSDDataset(train_data_path, processor)

sample = train_dataset[0]
print("Sample keys:", sample.keys())
print("Input IDs shape:", sample["input_ids"].shape)
print("Bbox shape:", sample["bbox"].shape)
print("Labels shape:", sample["labels"].shape)

Loaded 149 documents
Found 4 unique labels
Sample keys: KeysView({'input_ids': tensor([    0,   231, 39311,   163,  6361,  1296, 40135,   290,  1558, 18427,
         3367, 13675, 18103, 20857, 28120, 12435,   226, 23121,   163,  1499,
        32148, 42215,  6331,   857,   440,     4, 13675, 18103, 37604, 16092,
         2060,  6031, 10431,   234,    73,    83,  1698, 13718, 33267,   256,
        46368, 10081,  9834, 10704,  6015, 46937, 15770, 25938,   256,     4,
          226, 36935,    35,  2430,    19,   208,   466, 29997,     6,  1313,
          396,   208,   466, 29997,   131,   248,   254,   879,    35,  2430,
          396,   208,   466, 29997,     6,  1424,    12,   516,  1313,    19,
          208,   466, 29997,   111, 43997,  1272,    19,     5, 13675, 18103,
        20857, 28120, 12435, 28283,   256, 46368,  9715,    36,   347,  2118,
          347,  6597,  5161,   654,  8871,   195,     4,   973, 14708,  6936,
         9715, 14780, 24308,  1480, 16035,  2747, 15255,   741,

In [ ]:
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size]
)

print(f"Train size: {len(train_subset)}")
print(f"Validation size: {len(val_subset)}")

batch_size = 2

train_loader = DataLoader(
    train_subset, 
    batch_size=batch_size, 
    shuffle=True,
    collate_fn=lambda batch: {
        "input_ids": torch.stack([item["input_ids"] for item in batch]),
        "attention_mask": torch.stack([item["attention_mask"] for item in batch]),
        "bbox": torch.stack([item["bbox"] for item in batch]),
        "pixel_values": torch.stack([item["pixel_values"] for item in batch]),
        "labels": torch.stack([item["labels"] for item in batch])
    }
)

val_loader = DataLoader(
    val_subset, 
    batch_size=batch_size, 
    collate_fn=lambda batch: {
        "input_ids": torch.stack([item["input_ids"] for item in batch]),
        "attention_mask": torch.stack([item["attention_mask"] for item in batch]),
        "bbox": torch.stack([item["bbox"] for item in batch]),
        "pixel_values": torch.stack([item["pixel_values"] for item in batch]),
        "labels": torch.stack([item["labels"] for item in batch])
    }
)

for batch in train_loader:
    print("Batch shapes:")
    for key, value in batch.items():
        print(f"{key}: {value.shape}")
    break

Train size: 134
Validation size: 15
Batch shapes:
input_ids: torch.Size([2, 256])
attention_mask: torch.Size([2, 256])
bbox: torch.Size([2, 256, 4])
pixel_values: torch.Size([2, 3, 224, 224])
labels: torch.Size([2, 256])


In [ ]:
model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    num_labels=len(train_dataset.label2id),
    id2label=train_dataset.id2label,
    label2id=train_dataset.label2id
)
model.to(device)

print("Model initialized with:")
print(f" - Number of labels: {len(train_dataset.label2id)}")
print(f" - Model parameters: {sum(p.numel() for p in model.parameters()):,}")

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model initialized with:
 - Number of labels: 4
 - Model parameters: 125,330,052


In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR

epochs = 3
learning_rate = 5e-5

optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * epochs

from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR

warmup_scheduler = LinearLR(
    optimizer,
    start_factor=0.01,
    end_factor=1.0,
    total_iters=int(0.1 * total_steps)
)
    
main_scheduler = CosineAnnealingLR(
    optimizer,
    T_max=total_steps - int(0.1 * total_steps)  # Remaining steps
)

from torch.optim.lr_scheduler import SequentialLR
scheduler = SequentialLR(
    optimizer,
    schedulers=[warmup_scheduler, main_scheduler],
    milestones=[int(0.1 * total_steps)]
)

: 

In [ ]:
model.train()
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    total_loss = 0
    
    for batch in tqdm(train_loader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()
    
    avg_train_loss = total_loss / len(train_loader)
    print(f"Average training loss: {avg_train_loss:.4f}")
    
    model.eval()
    val_loss = 0
    for batch in tqdm(val_loader, desc="Validation"):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        val_loss += outputs.loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    print(f"Average validation loss: {avg_val_loss:.4f}")
    
    checkpoint_path = f"../models/trained/layoutlmv3-epoch-{epoch+1}"
    model.save_pretrained(checkpoint_path)
    processor.save_pretrained(checkpoint_path)
    print(f"Saved checkpoint to {checkpoint_path}")

print("Training complete!")


Epoch 1/3


Training:   0%|          | 0/67 [00:00<?, ?it/s]

/workspaces/Document_Layout_Understanding/venv/lib/python3.12/site-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
model_path = "../models/trained/layoutlmv3-funsd-final"
model.save_pretrained(model_path)
processor.save_pretrained(model_path)
print(f"Model saved to {model_path}")